# Exploratory Data Analysis for Steamit

### Importing Necessary Libraries

In [1]:
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import pandas as pd
import torch
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from transformers import BertTokenizer, BertForSequenceClassification

# Connect to the database
conn = sqlite3.connect('steam_games.db')

# Create a cursor object
cursor = conn.cursor()

# List all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Close the connection
conn.close()

# Print the list of tables
print("Tables in the database:", tables)


Tables in the database: [('game_details',), ('game_reviews',), ('sqlite_sequence',)]


In [2]:
# Correct table name based on your database structure
game_table = 'game_details'  # Replace with the actual table name if different
review_table = 'game_reviews'  # Replace with the actual table name if different

# Reconnect to the database
conn = sqlite3.connect('steam_games.db')

# Load the data into pandas DataFrames
games_df = pd.read_sql_query(f"SELECT * FROM {game_table}", conn)
reviews_df = pd.read_sql_query(f"SELECT * FROM {review_table}", conn)

# Close the connection
conn.close()

# Display the first few rows of the dataframes
games_df.head()



,appid,name,description,price,release_date,developer,publisher,tags
0,1462940,Ecto Portal,"The next generation of spooky displays, create...",₹ 529,"16 Jan, 2021",SpookyAction,SpookyAction,"Action, Adventure"
1,1462960,Evade Zero,Evade is a fast and intense local multiplayer ...,₹ 299,"18 Dec, 2020",Jonathan Francis,Self Published,"Action, Indie, Sports"
2,1462970,Jumpman (C64/MSDOS),"Run, climb and jump from platform to platform ...",₹ 155,"19 Nov, 2020","Epyx, Inc.",Pixel Games UK,"Action, Adventure, Simulation"
3,1462980,Dinosaurs Live Wallpaper: with Secret Sandboxes,,N/A,Coming soon,William at Oxford,William at Oxford,Utilities
4,1462990,Rushaug: Feline Warfare,A tactical platformer combining stealth and fa...,N/A,To be announced,Tobias Edvardsen,Tobias Edvardsen,"Action, Adventure"


In [3]:
reviews_df.columns

Index(['review_id', 'appid', 'review_text', 'voted_up', 'timestamp_created',
       'author_playtime_forever', 'author_playtime_last_two_weeks',
       'author_num_reviews'],
      dtype='object')

In [4]:
# Remove DLCs, Playtests, and Demos from games_df
filtered_games_df = games_df[~games_df['name'].str.contains("DLC|Playtest|Demo", case=False, na=False)]

# Function to clean and normalize text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

# Clean the review text
reviews_df['cleaned_text'] = reviews_df['review_text'].apply(clean_text)

# Function to check if text contains only English alphanumeric characters
def is_english_alphanumeric(text):
    return bool(re.match(r'^[a-z0-9\s]*$', text))

# Filter out rows with non-English or non-alphanumeric characters
reviews_df = reviews_df[reviews_df['cleaned_text'].apply(is_english_alphanumeric)]

# Display the cleaned data
print(reviews_df.head())

   review_id    appid                                        review_text  \
1          2  1491170      have fun playing this with the broken english   
3          4  1491180  Not very different from the first game in the ...   
4          5  1491180                           Better than the original   
5          6  1491180  As its predeseccor, it is a nice simplistic ad...   
6          7  1491190                                       best 5 bucks   

   voted_up  timestamp_created  author_playtime_forever  \
1         1         1635895805                      554   
3         1         1612534130                     1411   
4         1         1649783162                      542   
5         1         1621182874                      830   
6         1         1614101347                      609   

   author_playtime_last_two_weeks  author_num_reviews  \
1                               0                   5   
3                               0                 106   
4                

# Perform LDA for topic modeling

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Vectorize the cleaned text data
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
doc_term_matrix = vectorizer.fit_transform(reviews_df['cleaned_text'])

# Perform LDA for topic modeling
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(doc_term_matrix)

# Get the topic distributions for each review
topic_distributions = lda.transform(doc_term_matrix)

# Add the topic distributions to the DataFrame
reviews_df['topic_distribution'] = list(topic_distributions)


# Generate topic distributions

In [9]:
# Ensure topic distributions are numpy arrays and have consistent lengths
def pad_or_trim_distribution(distribution, length):
    if len(distribution) > length:
        return distribution[:length]
    elif len(distribution) < length:
        return np.pad(distribution, (0, length - len(distribution)), 'constant')
    return distribution

# Assume 10 topics for example
num_topics = 10

# Apply padding or trimming to topic distributions
reviews_df['topic_distribution'] = reviews_df['topic_distribution'].apply(lambda x: pad_or_trim_distribution(np.array(x), num_topics))

# Create user and item feature matrices
user_feature_matrix = reviews_df.groupby('author_num_reviews')['topic_distribution'].apply(lambda x: np.mean(list(x), axis=0)).values.tolist()
item_feature_matrix = reviews_df.groupby('appid')['topic_distribution'].apply(lambda x: np.mean(list(x), axis=0)).values.tolist()

# Convert lists to numpy arrays
user_feature_matrix = np.array(user_feature_matrix)
item_feature_matrix = np.array(item_feature_matrix)

# Transpose the item feature matrix
item_feature_matrix_transposed = item_feature_matrix.T

# Check the shapes after ensuring consistency
print("Shape of user_feature_matrix:", user_feature_matrix.shape)
print("Shape of item_feature_matrix:", item_feature_matrix.shape)
print("Shape of item_feature_matrix_transposed:", item_feature_matrix_transposed.shape)

# Perform the dot product
user_item_preference_matrix = np.dot(user_feature_matrix, item_feature_matrix_transposed)
print("Shape of user_item_preference_matrix:", user_item_preference_matrix.shape)

Shape of user_feature_matrix: (489, 10)
Shape of item_feature_matrix: (892, 10)
Shape of item_feature_matrix_transposed: (10, 892)
Shape of user_item_preference_matrix: (489, 892)


In [10]:
# Function to get top N recommendations for a given user index
def get_recommendations(user_index, user_item_matrix, item_ids, top_n=5):
    user_preferences = user_item_matrix[user_index]
    top_indices = user_preferences.argsort()[-top_n:][::-1]
    return [item_ids[i] for i in top_indices]

# Get the item IDs from the item_feature_matrix
item_ids = reviews_df['appid'].unique()

# Example: Get top 5 recommendations for user with index 0
user_index = 0
top_n = 5
recommended_items = get_recommendations(user_index, user_item_preference_matrix, item_ids, top_n)

print(f"Top {top_n} recommendations for user {user_index}:")
for item in recommended_items:
    print(item)

Top 5 recommendations for user 0:
1463500
1480970
1466400
1463130
1470940


In [11]:
# Function to get details of recommended items
def get_item_details(item_ids, games_df):
    available_columns = ['appid', 'name', 'description', 'price', 'release_date', 'developer', 'publisher', 'tags']
    details = games_df[games_df['appid'].isin(item_ids)][available_columns]
    return details

# Get details of the recommended items
recommended_items_details = get_item_details(recommended_items, games_df)

# Display the recommended items with details
print("Top 5 recommended games for user 0:")
print(recommended_items_details)


Top 5 recommended games for user 0:
        appid                                           name  \
14    1463130                                      Robobo TM   
19    1463500                             Conrun Virtual Gym   
201   1466400              Edge Of Dead: Under A Uranium Sky   
477   1470940  STAR FLEET II - Krellan Commander Version 2.0   
1168  1480970                               Brandins Buttons   

                                            description  price  release_date  \
14    Starts on slow gear and explore. Enter timed s...   ₹ 85  23 Jun, 2021   
19    Compete against AI in a virtual gym. Earn XP, ...  ₹ 199  12 Dec, 2020   
201   A bullet hell that let's you pickup and throw ...  ₹ 530  16 Nov, 2023   
477   Welcome to the future of space warfare! Star F...  ₹ 349  13 Nov, 2023   
1168  Challenge yourself with over 110 increasingly ...  ₹ 199  10 Dec, 2020   

                             developer                         publisher  \
14                    

In [12]:
np.save('item_feature_matrix.npy', item_feature_matrix)



In [13]:
np.save('item_feature_matrix.npy', user_feature_matrix)